In [16]:
import pandas as pd
from sklearn.linear_model  import LogisticRegression
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet')
from nltk.corpus import wordnet
from sklearn.model_selection import cross_val_score
df_comb=pd.read_csv("dis_sym_dataset_comb.csv")
df_norm=pd.read_csv("dis_sym_dataset_norm.csv")

X=df_comb.iloc[:,1:]
y=df_comb.iloc[:,0:1]

lr=LogisticRegression()
lr=lr.fit(X,np.ravel(y))
scores = cross_val_score(lr, X, np.ravel(y), cv=5)

dataset_symptoms=list(X.columns)
lemmatizer=WordNetLemmatizer()
splitter = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package wordnet to C:\Users\G Naveen
[nltk_data]     Chandra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Users\G Naveen Chandra\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [17]:
from bs4 import BeautifulSoup
import requests
def synonyms(term):
    synonyms = []
    
    for syn in wordnet.synsets(term):
        synonyms+=syn.lemma_names()
    return set(synonyms)

In [19]:
user_input=input("Enter symptoms by separating with commas\n").lower().split(",")
processed_symptoms=[]
for sym in user_input:
    sym=sym.replace("-"," ")
    sym=sym.replace("'","")
    sym=' '.join([lemmatizer.lemmatize(word) for word in splitter.tokenize(sym) ])
    processed_symptoms.append(sym)

print(processed_symptoms)

['sneezing']


In [20]:
from itertools import combinations

user_symptoms = []
for user_sym in processed_symptoms:
    user_sym = user_sym.split()
    str_sym = set()
    for comb in range(1, len(user_sym)+1):
        for subset in combinations(user_sym, comb):
            subset=' '.join(subset)
            subset = synonyms(subset) 
            str_sym.update(subset)
    str_sym.add(' '.join(user_sym))
    user_symptoms.append(' '.join(str_sym).replace('_',' '))
print("After query expansion done by using the symptoms entered")


After query expansion done by using the symptoms entered


In [21]:

found_symptoms = set()
for idx, data_sym in enumerate(dataset_symptoms):
    data_sym_split=data_sym.split()
    for user_sym in user_symptoms:
        count=0
        for symp in data_sym_split:
            if symp in user_sym.split():
                count+=1
        if count/len(data_sym_split)>0.5:
            found_symptoms.add(data_sym)
found_symptoms = list(found_symptoms)
print(found_symptoms)

['sneezing']


In [22]:
print("Top matching symptoms from your search!")
for idx, symp in enumerate(found_symptoms):
    print(idx,":",symp)

Top matching symptoms from your search!
0 : sneezing


In [23]:
select=input("Enter symptoms with index number : ").split()
print(select)
sym_matched=[]
dis_list=set()
for id in select:
    sym_matched.append(found_symptoms[int(id)])
    sym=found_symptoms[int(id)]
    dis_list.update((df_norm[df_norm[sym]==1]['label_dis']))
print(sym_matched)
print(dis_list)

['0']
['sneezing']
{'Perennial Allergic Conjunctivitis'}


Listing Co-Occuring Symptoms

In [24]:
counter_list=[]
for dis in dis_list:
    row=df_norm[df_norm['label_dis']==dis].values.tolist()
    row[0].pop(0)
    for id,x in enumerate(row[0]):
        if x!=0 and df_norm.columns[id+1] not in sym_matched:
            counter_list.append(df_norm.columns[id+1])
counter_list=set(counter_list)
counter_list=list(counter_list)
print(counter_list)





['watery eye', 'swelling around eye', 'red', 'itchy', 'stuffy itchy nose', 'itchy ear']


Displaying only 5 at a time and skipping if user enters -1 and stopping if user enters 'no'

In [26]:
count=0
for id,val in enumerate(counter_list):
    count+=1
    if count==5:
        print("Select symptoms: ")
        for i in range(5):
            
            print(id-(4-i),":",counter_list[id-(4-i)])
        user_input_sym=input("Enter the symptoms index (or) enter -1 to skip (or) enter '-2' to stop").split()
        if len(user_input_sym)!=1:
         for x in user_input_sym[:-1]:
            sym_matched.append(counter_list[int(x)])
        
        if int(user_input_sym[-1])==-1:
            count=0
            continue
        elif int(user_input_sym[-1])==-2:
            break
        
print(sym_matched)



Select symptoms: 
0 : watery eye
1 : swelling around eye
2 : red
3 : itchy
4 : stuffy itchy nose
['sneezing', 'stuffy itchy nose']


In [27]:
from sklearn.model_selection import train_test_split, cross_val_score
sample=[]
for i in range(len(df_norm.columns)-1):
    sample.append(0)
for id,val in enumerate(dataset_symptoms):
    for sym in sym_matched:
        if sym==val:
            sample[id]=1
print(sample)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

PREDICT DISEASE

In [28]:
'''prediction = lr.predict_proba([sample])
print(prediction)'''
'''Show top k diseases and their probabilities to the user.

K in this case is 10'''


'''k = 10
diseases = list(set(y['label_dis']))
diseases.sort()
topk = prediction[0].argsort()[-k:][::-1]
print(topk)'''

prediction=lr.predict_proba([sample])
diseases=list(set(y['label_dis']))
diseases.sort()
k=10
topk=prediction[0].argsort()[-k:][::-1]


c:\Users\G Naveen Chandra\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


NameError: name 'diseases' is not defined

In [29]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, cross_val_score
from statistics import mean
from nltk.corpus import wordnet 
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from itertools import combinations
from time import time
from collections import Counter
import operator

In [30]:
import math
print(f"\nTop {k} diseases predicted based on symptoms")
topk_dict = {}
# Show top 10 highly probable disease to the user.
for idx,t in  enumerate(topk):
    match_sym=set()
    row = df_norm.loc[df_norm['label_dis'] == diseases[t]].values.tolist()
    row[0].pop(0)

    for idx,val in enumerate(row[0]):
        if val!=0:
            match_sym.add(dataset_symptoms[idx])
    prob = (len(match_sym.intersection(set(sym_matched)))+1)/(len(set(match_sym))+1)
    prob *= mean(scores)
    topk_dict[t] = prob
j = 0
topk_index_mapping = {}
topk_sorted = dict(sorted(topk_dict.items(), key=lambda kv: kv[1], reverse=True))
for key in topk_sorted:
  prob = topk_sorted[key]*100
  print(str(j) + " Disease name:",diseases[key], "\tProbability:",str(round(prob, 2))+"%")
  topk_index_mapping[j] = key
  j += 1



Top 10 diseases predicted based on symptoms
0 Disease name: Perennial Allergic Conjunctivitis 	Probability: 33.45%
1 Disease name: Mouth Breathing 	Probability: 25.48%
2 Disease name: Breast Cancer / Carcinoma 	Probability: 12.74%
3 Disease name: Scrapie 	Probability: 11.15%
4 Disease name: Celiacs disease 	Probability: 11.15%
5 Disease name: Carpal Tunnel Syndrome 	Probability: 11.15%
6 Disease name: Rabies 	Probability: 9.91%
7 Disease name: Hyperthyroidism 	Probability: 9.91%
8 Disease name: Polycystic ovary syndrome (PCOS) 	Probability: 8.92%
9 Disease name: Myocardial Infarction (Heart Attack) 	Probability: 7.43%


In [28]:
import html_table_parser.parser_functions
import urllib

In [1]:
from bs4 import BeautifulSoup
import requests
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pandas as pd

In [2]:
import lxml
url="https://en.wikipedia.org/wiki/diseases[topk_index_mapping[0]]"
response=requests.get(url=url)
soup=BeautifulSoup(response.content,'html.parser')
dis_data=soup.find('table',{'class':'infobox'})
df=pd.read_html(str(dis_data))
df=pd.DataFrame(df[0])
df=df.dropna()
df.drop(2,axis=0,inplace=True)
print(df)

ValueError: No tables found

In [26]:
import collections
collections.Callable = collections.abc.Callable